# Homework 2 - Experiment Tracking & Model Management

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and
model management.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [14]:
import mlflow

print(mlflow.__version__)

2.22.0


## Q2. Download and preprocess the data

We'll use the Yellow Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework2) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [10]:
%run preprocess_data.py --raw_data_path ../data --dest_path ./output

In [12]:
from pathlib import Path

output_dir = Path('./output')

file_count = len([f for f in output_dir.iterdir() if f.is_file()])
print(f"Number of files: {file_count}")

Number of files: 4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework2).

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [15]:
home_path_str = Path.home()
backend_store_uri = f'sqlite:///{home_path_str}/mlflow/backend/mlflow.db'
artifact_location = f'file:{home_path_str}/mlflow/mlruns'

# Setting backend store
mlflow.set_tracking_uri(uri=backend_store_uri)

# Setting experiment configuration (name and artifact location)
experiment_name = "homework2-experiment"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(
        name=experiment_name,
        artifact_location=artifact_location
    )
mlflow.set_experiment(experiment_name=experiment_name)

<Experiment: artifact_location='file:/home/jxareas/mlflow/mlruns', creation_time=1748332878823, experiment_id='3', last_update_time=1748332878823, lifecycle_stage='active', name='homework2-experiment', tags={}>

In [24]:
%run train.py

**Homework 2 Random Forest Logged Params** (with `min_samples_split`):

![](../images/hw2-rf-logged-params.png)

Hence, the value for `min_samples_split` is 2, as shown in the image.

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

The `default-artifact-root`, as shown by script
```bash
BACKEND_STORE_URI="sqlite:///$HOME/mlflow/backend/mlflow.db"
ARTIFACT_ROOT="$HOME/mlflow/artifacts"

# Start MLflow UI
mlflow ui \
  --backend-store-uri "$BACKEND_STORE_URI" \
  --default-artifact-root "$ARTIFACT_ROOT"
```

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`.
We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336


In [9]:
%run hpo.py

🏃 View run handsome-hog-164 at: http://127.0.0.1:5000/#/experiments/4/runs/95c51a788e4b4896943c27583bbe9228

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4

🏃 View run salty-dove-789 at: http://127.0.0.1:5000/#/experiments/4/runs/89e6bfcb410f4559ace61881fdfe052f

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4                   

🏃 View run fun-boar-141 at: http://127.0.0.1:5000/#/experiments/4/runs/cdb8de5f8aa540fbaaba9b4739d980a0

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4                   

🏃 View run sincere-shoat-605 at: http://127.0.0.1:5000/#/experiments/4/runs/9102bd5902c74f2fbc2dad64d6f66325

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4                   

🏃 View run legendary-gnat-4 at: http://127.0.0.1:5000/#/experiments/4/runs/27c4bed41e484af18a05de0bd22d9745

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4                   

🏃 View run valuable-fish-486 at: http://127.0.0.1:5000/#/experiments/4/runs/0cb

Random Forest HyperOpt Descending RMSE:

![](../images/hw2-rf-hyperopt-desc-rmse.png)

The best validation RMSE gotten in the trials is `5.335`.

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production.
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs.
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [11]:
%run register_model.py

2025/05/27 02:40:36 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.


🏃 View run welcoming-wren-173 at: http://127.0.0.1:5000/#/experiments/5/runs/b08ba25ef4604a8e89d445786475dadf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
🏃 View run placid-mare-695 at: http://127.0.0.1:5000/#/experiments/5/runs/6085cc78cfc94202accb56f8bc649007
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
🏃 View run lyrical-pug-814 at: http://127.0.0.1:5000/#/experiments/5/runs/bdc2e9cef16d42c4815493f4bdaf1b00
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
🏃 View run nervous-rat-246 at: http://127.0.0.1:5000/#/experiments/5/runs/69dc6c32447841e08f89b6f163cd9c15
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


Successfully registered model 'random-forest-model'.
2025/05/27 02:41:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-model, version 1


🏃 View run stylish-zebra-547 at: http://127.0.0.1:5000/#/experiments/5/runs/93349ec6e3b240dcb8c3b269c64d925f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


Created version '1' of model 'random-forest-model'.


![](../images/hw2-rf-best-models-test-rmse.png)

Test RMSE for the selected Random Forest model is `5.57`.